In [1]:
%pip install mpqp

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.9 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/162.6 kB ? eta -:--:--
     ---------------------- ---------------- 92.2/162.6 kB 2.6 MB/s eta 0:00:01
     ----------------------------------- -- 153.6/162.6 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 162.6/162.6 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maxim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [51]:
from mpqp.gates import *
from mpqp.execution.result import Result
from mpqp import QCircuit, Barrier
from mpqp.execution import run, IBMDevice
from math import floor
import numpy as np


class InverseQFT(QCircuit):
      
    def __init__(self,n_qubits):
        super().__init__(n_qubits, nb_cbits=n_qubits)
        self._build()
    
    def _build(self):
        self.add([SWAP(i, self.nb_qubits - 1 - i)
            for i in range(int(floor(self.nb_qubits / 2)))])
        j = self.nb_qubits - 1
        while j >= 0:
            self.add(Barrier())
            self.add([CRk(i+1 - j, i, j).inverse() for i in range(j+1, self.nb_qubits)])
            self.add(H(j))
            j -= 1

class QFT(QCircuit):
      
    def __init__(self,n_qubits,inverse=False):
        
        super().__init__(n_qubits, nb_cbits=n_qubits)
        self.inverse = inverse
        self._build()
    
    def _build(self):
        for j in range(self.nb_qubits):
            self.add(H(j))
            self.add([CRk(i+1 - j, i, j) for i in range(j+1, self.nb_qubits)])
            self.add(Barrier())
        self.add([SWAP(i, self.nb_qubits-1-i) for i in range(int(floor(self.nb_qubits / 2)))])
        
        if self.inverse == True:
           self.inverse()

In [52]:
def demo(n_qubits) -> None:
    qft = QFT(n_qubits)
    iqft = InverseQFT(n_qubits)
    combo = qft + iqft

    print("\nInverse QFT")
    print(iqft)
    print("\nForward QFT + Inverse QFT")
    print(combo)

    result = run(combo, IBMDevice.AER_SIMULATOR_STATEVECTOR)
    print("\nExecution results:")
    print(result)

demo(n_qubits=7)


Inverse QFT
               ░       ░                 ░                           ░ »
q_0: ─X────────░───────░─────────────────░───────────────────────────░─»
      │        ░       ░                 ░                           ░ »
q_1: ─┼──X─────░───────░─────────────────░───────────────────────────░─»
      │  │     ░       ░                 ░                           ░ »
q_2: ─┼──┼──X──░───────░─────────────────░───────────────────────────░─»
      │  │  │  ░       ░                 ░                           ░ »
q_3: ─┼──┼──┼──░───────░─────────────────░───────────────────────────░─»
      │  │  │  ░       ░                 ░                     ┌───┐ ░ »
q_4: ─┼──┼──X──░───────░─────────────────░──■─────────■────────┤ H ├─░─»
      │  │     ░       ░           ┌───┐ ░  │P(-π/2)  │        └───┘ ░ »
q_5: ─┼──X─────░───────░──■────────┤ H ├─░──■─────────┼──────────────░─»
      │        ░ ┌───┐ ░  │P(-π/2) └───┘ ░            │P(-π/4)       ░ »
q_6: ─X────────░─┤ H ├─░──■───────────